In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.SparkContext  
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._

import $ivy.$                                   

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@ea54de1
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSet

In [2]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@350f8b81
defined function sc
import spark.implicits._

In [4]:
val frame = spark.read.option("header", true).option("inferSchema",true).csv("picsart_1.csv")

frame: DataFrame = [country_code: string, device_id: string ... 12 more fields]

In [5]:
val df = frame.drop("advertising_id", "timezone", "duration", "app", "v").sample(true,0.1).withColumn("date", frame("timestamp").cast(DateType))

df: DataFrame = [country_code: string, device_id: string ... 8 more fields]

In [6]:
val df1 = df.select("device_id", "event_type", "timestamp", "date")

df1: DataFrame = [device_id: string, event_type: string ... 2 more fields]

In [ ]:
//2.Calculate daily active users trend over month
df1.groupBy("date").agg(countDistinct(col("device_id"))).sort(col("date")).show(30)

In [21]:
// 3. Calculate weekly seasonality
val df2 = df1.withColumn("week_day", date_format(col("timestamp"), "E"))
df2.groupBy("week_day").count().sort(col("count").desc).show()

df2: DataFrame = [device_id: string, event_type: string ... 3 more fields]

In [18]:
//5.Calculate editor_open -> editor_done_click -> photo_upload funnel
val sum_count_editor_open = df1.where(col("event_type")==="editor_open").groupBy("date").agg(countDistinct(col("device_id")).as("count_editor_open")).
                            agg(sum("count_editor_open"))show()
val sum_count_editor_done_click = df1.where(col("event_type")==="editor_done_click").groupBy("date").agg(countDistinct(col("device_id")).as("count_editor_done_click")).
                            agg(sum("count_editor_done_click"))show()
val sum_count_photo_upload = df1.where(col("event_type")==="photo_upload").groupBy("date").agg(countDistinct(col("device_id")).as("count_photo_upload")).
                            agg(sum("count_photo_upload"))show()

+----------------------+
|sum(count_editor_open)|
+----------------------+
|                   103|
+----------------------+

+----------------------------+
|sum(count_editor_done_click)|
+----------------------------+
|                          49|
+----------------------------+

+-----------------------+
|sum(count_photo_upload)|
+-----------------------+
|                     46|
+-----------------------+

